In [11]:
from visual_behavior.visualization.qc import session_plots as sp
from visual_behavior.visualization.qc import container_plots as cp
from visual_behavior.visualization.qc import data_loading as dl
from visual_behavior.visualization.qc import data_processing as dp
import visual_behavior.utilities as vbu

from visual_behavior.translator.foraging2 import data_to_change_detection_core
from visual_behavior.translator.core import create_extended_dataframe
from visual_behavior.change_detection.trials import summarize
import visual_behavior.database as db

%matplotlib inline
%widescreen
%standard_imports

imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


## get ophys experiment table

In [2]:
ophys_experiment_table = dl.get_filtered_ophys_experiment_table().sort_values(by='date_of_acquisition', ascending=False).reset_index()


In [49]:
container_df = dl.build_container_df()
len(container_df)

43

In [3]:
ophys_experiment_table.sample(5)

,index,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at
99,2376,911149119,910971181,911025799,906963896,VisualBehaviorTask1B,container_qc,passed,20190723_462544_1imagesB,OPHYS_1_images_B,CAM2P.3,2019-07-23 20:06:37.000000,876289252,866070199,M,123.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375.0,VISp,NaN
174,738,848697604,848143026,848348215,845588020,VisualBehavior,container_qc,passed,20190409_445270_1imagesA,OPHYS_1_images_A,CAM2P.3,2019-04-09 16:16:45.000000,828775962,820871408,M,96.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375.0,VISp,NaN
151,367,863735602,863527124,863610582,848390542,VisualBehavior,container_qc,passed,20190506_447663_2imagesA,OPHYS_2_images_A_passive,CAM2P.4,2019-05-06 20:07:55.000000,832570530,823826986,F,114.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175.0,VISp,NaN
109,370,901559828,900802327,901251663,895259496,VisualBehaviorTask1B,container_qc,passed,20190708_461946_3imagesB,OPHYS_3_images_B,CAM2P.5,2019-07-08 16:09:58.000000,872873989,862671211,F,111.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175.0,VISp,NaN
178,2128,848694639,845219209,845441512,843049827,VisualBehavior,container_qc,passed,20190403_440298_4imagesB,OPHYS_4_images_B,CAM2P.3,2019-04-03 15:55:13.000000,822459611,813703544,M,114.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375.0,VISp,NaN


In [6]:
len(ophys_experiment_table)

247

## functions

In [139]:
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc
manifest_file = r"//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020_cache/covid_cache/manifest.json"
cache = bpc.from_lims(manifest=manifest_file)
all_experiments = cache.get_experiment_table() 
all_experiments = all_experiments.reset_index()

def oeid_to_uuid(oeid):
    return db.convert_id({'ophys_experiment_id':oeid},'behavior_session_uuid')

def get_stage_from_mongo(ophys_experiment_id):
    uuid = oeid_to_uuid(ophys_experiment_id)
    vb = db.Database('visual_behavior_data')
    stats = vb.behavior_data['summary'].find_one({'behavior_session_uuid':uuid})
    vb.close()
    if stats is None or ('error_on_load' in stats.keys() and stats['error_on_load'] == 1):
        return None
    else:
        return stats['stage']

def get_date_from_mongo(ophys_experiment_id):
    uuid = oeid_to_uuid(ophys_experiment_id)
    vb = db.Database('visual_behavior_data')
    stats = vb.behavior_data['summary'].find_one({'behavior_session_uuid':uuid})
    vb.close()
    if stats is None or ('error_on_load' in stats.keys() and stats['error_on_load'] == 1):
        return None
    else:
        return stats['startdatetime']
    
def get_container_info(ophys_experiment_id):
    container_id = all_experiments.query('ophys_experiment_id == @ophys_experiment_id')['container_id'].iloc[0]
    return dp.ophys_container_info_df(container_id)

def get_lims_stage_name(ophys_experiment_id):
    container_info = get_container_info(ophys_experiment_id)
    try:
        return container_info.query('ophys_experiment_id == @ophys_experiment_id')['stage_name_lims'].iloc[0]
    except:
        pass

def get_mtrain_stage_name(ophys_experiment_id):
    container_info = get_container_info(ophys_experiment_id)
    try:
        return container_info.query('ophys_experiment_id == @ophys_experiment_id')['stage_name_mtrain'].iloc[0]
    except:
        pass
    
    
def rewrite_record(uuid):
    try:
        pkl_path = db.get_pkl_path(uuid)
        data = pd.read_pickle(pkl_path)
        vb = db.Database('visual_behavior_data')
        core_data = data_to_change_detection_core(data)
        trials = create_extended_dataframe(**core_data).drop(columns=['date', 'LDT_mode'])
        summary = summarize.session_level_summary(trials).iloc[0].to_dict()
        summary.update({'error_on_load':0})
        db.update_or_create(
            vb['behavior_data']['summary'],
            db.simplify_entry(summary),
            ['behavior_session_uuid'],
            force_write=False
        )
        vb.close()
    except:
        print('failed on uuid {}'.format(uuid))

## add columns with date and session from mongo (which comes directly from PKL files)

In [37]:
ophys_experiment_table['mongo_stage_name'] = ophys_experiment_table['ophys_experiment_id'].map(lambda oeid:get_stage_from_mongo(oeid))
ophys_experiment_table['mongo_date'] = ophys_experiment_table['ophys_experiment_id'].map(lambda oeid:get_date_from_mongo(oeid))
ophys_experiment_table['uuid'] = ophys_experiment_table['ophys_experiment_id'].map(lambda oeid:oeid_to_uuid(oeid))

ophys_experiment_table['lims_stage_name'] = ophys_experiment_table['ophys_experiment_id'].map(lambda oeid:get_lims_stage_name(oeid))
ophys_experiment_table['mtrain_stage_name'] = ophys_experiment_table['ophys_experiment_id'].map(lambda oeid:get_mtrain_stage_name(oeid))

In [38]:
ophys_experiment_table.sample(5)

,index,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,mongo_session_name,mongo_date,uuid,mongo_stage_name,lims_stage_name,mtrain_stage_name
74,225,939327156,938175546,938425882,930022332,VisualBehavior,container_qc,passed,20190904_459777_1ImagesA,OPHYS_1_images_A,CAM2P.3,2019-09-04 15:26:03.000000,863814878,854625263,F,180.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175.0,VISp,NaN,OPHYS_1_images_A,2019-09-04 16:25:56.552,7a4813c4-0296-4a7b-ba87-6c9cc3a7b76f,OPHYS_1_images_A,OPHYS_1_images_A,OPHYS_1_images_A
231,2785,795948257,795866393,795903435,814796558,VisualBehavior,container_qc,passed,20181214_416369_2P5_VISp_375_3,OPHYS_3_images_A,CAM2P.5,2018-12-15 00:07:47.000000,761346702,756577249,F,130.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",375.0,VISp,NaN,OPHYS_3_images_A,2018-12-15 00:32:14.278,42600851-57b9-40da-8303-0374c31bd6d1,OPHYS_3_images_A,OPHYS_3_images_A,OPHYS_3_images_A
161,358,853328115,852794147,852966248,848390542,VisualBehavior,container_qc,passed,20190417_447663_3imagesA,OPHYS_3_images_A,CAM2P.4,2019-04-17 15:54:23.000000,832570530,823826986,F,95.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175.0,VISp,NaN,OPHYS_3_images_A,2019-04-17 16:15:38.810,3481b051-c9ee-4e7a-9077-907d6dce2e56,OPHYS_3_images_A,OPHYS_3_images_A,OPHYS_3_images_A
126,1454,889771676,888940531,889175194,890047007,VisualBehavior,container_qc,passed,20190618_456916_Images1A,OPHYS_1_images_A,CAM2P.5,2019-06-18 15:04:23.000000,855988827,847074148,F,117.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175.0,VISp,NaN,OPHYS_1_images_A,2019-06-18 16:17:59.499,f1018322-3c3b-4b1e-98b8-f169dd454f4b,OPHYS_1_images_A,OPHYS_1_images_A,OPHYS_1_images_A
218,2682,807753318,807055274,807198458,814797528,VisualBehavior,container_qc,passed,20180111_421136_5imagesB,OPHYS_5_images_B_passive,CAM2P.5,2019-01-11 16:11:30.000000,767468166,760949544,M,134.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Vip-IRES-Cre],175.0,VISp,NaN,OPHYS_5_images_B_passive,2019-01-11 16:50:36.621,02f1e796-f1d2-4ba5-bacc-d246ca7d4e36,OPHYS_5_images_B_passive,OPHYS_5_images_B_passive,OPHYS_5_images_B_passive


## search for mismatches in stage name (there are  none)

In [175]:
cols = ['ophys_experiment_id','container_id','project_code','container_workflow_state','experiment_workflow_state','session_name','session_type','mongo_stage_name','lims_stage_name','mtrain_stage_name']
ophys_experiment_table.query('session_type != mtrain_stage_name')[cols]

,ophys_experiment_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,mongo_stage_name,lims_stage_name,mtrain_stage_name
9,974384470,968451510,VisualBehavior,container_qc,passed,20191030_479426_3imagesAretake,OPHYS_3_images_A,OPHYS_3_images_A,OPHYS_3_images_A,NaN
242,788490510,782536745,VisualBehavior,container_qc,passed,20181129_403491_6imagesB,OPHYS_6_images_B,OPHYS_6_images_B,OPHYS_6_images_B,OPHYS_7_receptive_field_mapping


In [44]:
len(ophys_experiment_table)

247

In [34]:
get_mtrain_stage_name(896160394)

'OPHYS_4_images_B'

In [52]:
np.sort(dl.get_filtered_ophys_experiment_table()['session_type'].unique())

array(['OPHYS_1_images_A', 'OPHYS_1_images_B', 'OPHYS_2_images_A_passive',
       'OPHYS_2_images_B_passive', 'OPHYS_3_images_A', 'OPHYS_3_images_B',
       'OPHYS_4_images_A', 'OPHYS_4_images_B', 'OPHYS_5_images_A_passive',
       'OPHYS_5_images_B_passive', 'OPHYS_6_images_A', 'OPHYS_6_images_B'],
      dtype=object)

In [53]:
new_manifest_path = r'\\allen\programs\braintv\workgroups\nc-ophys\visual_behavior\2020_cache\covid_cache'.replace('\\','/')
manifest = 

'//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020_cache/covid_cache'

In [101]:
visb_experiments = all_experiments[all_experiments.project_code.isin(['VisualBehavior','VisualBehaviorTask1B'])].copy()
len(visb_experiments)

827

In [141]:
visb_experiments['mongo_stage_name'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_stage_from_mongo(oeid))
visb_experiments['mongo_date'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_date_from_mongo(oeid))
visb_experiments['uuid'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:oeid_to_uuid(oeid))

visb_experiments['lims_stage_name'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_lims_stage_name(oeid))
visb_experiments['mtrain_stage_name'] = visb_experiments['ophys_experiment_id'].map(lambda oeid:get_mtrain_stage_name(oeid))

In [176]:
visb_experiments[pd.isnull(visb_experiments['session_type'])][cols]

,ophys_experiment_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,mongo_stage_name,lims_stage_name,mtrain_stage_name
70,868245889,898954413,VisualBehavior,failed,failed,20190514_448897_7RF,NaN,None,None,NaN
74,872434950,898954413,VisualBehavior,failed,failed,20190520_448897_7RF_Map,NaN,None,None,NaN
393,894725344,902748564,VisualBehavior,container_qc,failed,20190625_456564_7RF,NaN,None,None,NaN
396,897374358,902748564,VisualBehavior,container_qc,failed,20190628_456564_7RF,NaN,None,None,NaN
605,910199584,895259658,VisualBehaviorTask1B,failed,passed,20190719_460746_7RF_map,NaN,None,None,NaN
608,972666750,966316564,VisualBehaviorTask1B,failed,created,20191023_479835_session1b,NaN,OPHYS_1_images_B,None,OPHYS_1_images_B
823,853965797,845588020,VisualBehavior,container_qc,failed,20190417_445270_7RF,NaN,None,None,NaN
831,862016695,845588020,VisualBehavior,container_qc,failed,20190502_445270_7RF,NaN,None,None,NaN
833,864936381,845588020,VisualBehavior,container_qc,failed,20190508_445270_7RF,NaN,None,None,NaN
909,928024651,910170512,VisualBehaviorTask1B,failed,qc,20190820_467190_7receptivefield,NaN,None,None,NaN


In [174]:
visb_experiments

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,mongo_stage_name,mongo_date,uuid,lims_stage_name,mtrain_stage_name
64,862023618,861562000,861701835,898954413,VisualBehavior,failed,passed,20190502_448897_1imagesA,OPHYS_1_images_A,CAM2P.5,2019-05-02 15:17:15.000000,835602467,827778608,M,108.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_1_images_A,2019-05-02 15:37:53.020,7eb9cdb9-3c2e-49be-b5c9-b644beba71dd,OPHYS_1_images_A,OPHYS_1_images_A
65,862848084,862179326,862326313,898954413,VisualBehavior,failed,passed,20190503_448897_2imagesA,OPHYS_2_images_A_passive,CAM2P.5,2019-05-03 14:56:51.000000,835602467,827778608,M,109.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_2_images_A_passive,2019-05-03 15:36:43.056,443e6aa3-1428-41c6-9aac-01ad5cce964d,OPHYS_2_images_A_passive,OPHYS_2_images_A_passive
66,863736179,863325716,863376857,898954413,VisualBehavior,failed,failed,20190506_448897_3imagesA,OPHYS_3_images_A,CAM2P.5,2019-05-06 14:52:24.000000,835602467,827778608,M,112.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_3_images_A,2019-05-06 15:34:23.660,d5b7d701-6f27-4da0-b130-4bc0cffb688c,OPHYS_3_images_A,OPHYS_3_images_A
67,865748127,865019326,865164268,898954413,VisualBehavior,failed,failed,20190509_448897_4imagesB,OPHYS_4_images_B,CAM2P.5,2019-05-09 14:50:09.000000,835602467,827778608,M,115.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_4_images_B,2019-05-09 15:41:34.809,92e89983-cf00-45d4-b501-5aae799bed49,OPHYS_4_images_B,OPHYS_4_images_B
68,866463736,865847948,865942649,898954413,VisualBehavior,failed,passed,20190510_448897_5imagesB,OPHYS_5_images_B_passive,CAM2P.5,2019-05-10 14:51:24.000000,835602467,827778608,M,116.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_5_images_B_passive,2019-05-10 15:38:13.163,b2651ad4-98b4-49bc-bf9a-07f4ae3fc6e8,OPHYS_5_images_B_passive,OPHYS_5_images_B_passive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,903405627,902777219,902871565,895259496,VisualBehaviorTask1B,container_qc,passed,20190710_461946_4imagesA,OPHYS_4_images_A,CAM2P.5,2019-07-10 17:19:50.000000,872873989,862671211,F,113.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_4_images_A,2019-07-10 17:40:14.450,12e7ab74-0c87-44e0-ac72-422a8712c6e7,OPHYS_4_images_A,OPHYS_4_images_A
3026,904155155,903711544,903842232,895259496,VisualBehaviorTask1B,container_qc,passed,20190711_461946_5imagesApassive,OPHYS_5_images_A_passive,CAM2P.5,2019-07-11 17:07:58.000000,872873989,862671211,F,114.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_5_images_A_passive,2019-07-11 17:34:55.273,fd5438ca-470a-45ab-9774-ff571604a88f,OPHYS_5_images_A_passive,OPHYS_5_images_A_passive
3027,905551719,904554819,904844395,895259496,VisualBehaviorTask1B,container_qc,failed,20190712_461946_6imagesA,OPHYS_6_images_A,CAM2P.5,2019-07-12 16:51:10.000000,872873989,862671211,F,115.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,[Ai93(TITL-GCaMP6f)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_6_images_A,2019-07-12 19:24:32.146,681f0fa3-2aa8-40b0-a2d3-6ac33a008310,OPHYS_6_images_A,OPHYS_6_images_A
3028,906877227,906303646,906623513,895259496,VisualBehaviorTask1B,container_qc,passed,20190715_4

In [177]:
my_mismatches = visb_experiments.query('session_type != mongo_stage_name')[cols]
my_mismatches

,ophys_experiment_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,mongo_stage_name,lims_stage_name,mtrain_stage_name
70,868245889,898954413,VisualBehavior,failed,failed,20190514_448897_7RF,NaN,None,None,NaN
74,872434950,898954413,VisualBehavior,failed,failed,20190520_448897_7RF_Map,NaN,None,None,NaN
86,996046242,991913453,VisualBehaviorTask1B,completed,passed,20191223_476970_7RF,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping,NaN
93,797257159,791352433,VisualBehavior,container_qc,passed,20181217_412036_7_RF_map,full_field_test,None,full_field_test,NaN
103,834279512,825183217,VisualBehavior,container_qc,passed,20190307_429952_7RF,VisCodingTargetedMovieClips,None,VisCodingTargetedMovieClips,NaN
116,885061426,876693090,VisualBehavior,container_qc,passed,20190611_449441_7RF_Map,VisCodingTargetedMovieClips,None,VisCodingTargetedMovieClips,NaN
126,995382231,991913064,VisualBehaviorTask1B,holding,failed,20191218_491060_7RF,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping,NaN
127,995623020,991913064,VisualBehaviorTask1B,holding,qc,20191220_491060_7RF_retake,OPHYS_7_receptive_field_mapping,None,OPHYS_7_receptive_field_mapping,NaN
152,881881193,869781307,VisualBehavior,container_qc,passed,20190605_450471_7RF,VisCodingTargetedMovieClips,None,VisCodingTargetedMovieClips,NaN
223,872432459,863992815,VisualBehavior,container_qc,passed,20190520_448900_7RF,VisCodingTargetedMovieClips,None,VisCodingTargetedMovieClips,NaN


In [98]:
visb_experiments['session_type'].unique()

array(['OPHYS_1_images_A', 'OPHYS_2_images_A_passive', 'OPHYS_3_images_A',
       'OPHYS_4_images_B', 'OPHYS_5_images_B_passive', 'OPHYS_6_images_B',
       nan, 'OPHYS_1_images_B', 'OPHYS_2_images_B_passive',
       'OPHYS_3_images_B', 'OPHYS_4_images_A', 'OPHYS_6_images_A',
       'OPHYS_5_images_A_passive', 'OPHYS_7_receptive_field_mapping',
       'full_field_test', 'VisCodingTargetedMovieClips',
       'OPHYS_0_images_A_habituation'], dtype=object)

In [104]:
visb_experiments['mongo_stage_name'].unique()

array(['OPHYS_1_images_A', 'OPHYS_2_images_A_passive', 'OPHYS_3_images_A',
       'OPHYS_4_images_B', 'OPHYS_5_images_B_passive', 'OPHYS_6_images_B',
       None, 'OPHYS_1_images_B', 'OPHYS_2_images_B_passive',
       'OPHYS_3_images_B', 'OPHYS_4_images_A', 'OPHYS_6_images_A',
       'OPHYS_5_images_A_passive', 'OPHYS_0_images_A_habituation',
       'OPHYS_3_images_E'], dtype=object)

In [185]:
container_manifest = dl.build_container_df()
container_list = container_manifest["container_id"].unique()

all_containers_df = pd.DataFrame()
for ophys_container_id in container_list:
    container_df =  dp.ophys_container_info_df(ophys_container_id)
    all_containers_df = all_containers_df.append(container_df)
all_containers_df = all_containers_df.reset_index(drop = True)

possible_stage_names_lims = all_containers_df["stage_name_lims"].unique()
possible_stage_names_mtrain = all_containers_df["stage_name_mtrain"].unique()
# print("POSSIBLE STAGE NAMES LIMS: " + str(possible_stage_names_lims))

# print("POSSIBLE STAGE NAMES MTRAIN: " + str(possible_stage_names_mtrain))

stage_name_subset = all_containers_df[["ophys_container_id",
                                       "ophys_experiment_id", 
                                       "ophys_session_id", 
                                       "experiment_workflow_state", 
                                       "stage_name_lims", 
                                       "stage_name_mtrain"]].copy()

mismatch_df = stage_name_subset.loc[stage_name_subset["stage_name_lims"]!= stage_name_subset["stage_name_mtrain"]].copy().reset_index(drop=True)
mismatch_df.to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/stage_name_mismatches_identified_by_kate_03.21.2020.csv', index=False)

In [150]:
mismatch_df

,ophys_container_id,ophys_experiment_id,ophys_session_id,experiment_workflow_state,stage_name_lims,stage_name_mtrain
0,978206330,990662141,990457777,passed,OPHYS_7_receptive_field_mapping,NaN
1,969421516,967007345,966317904,passed,OPHYS_7_receptive_field_mapping,NaN
2,968451510,974384470,974019098,passed,OPHYS_3_images_A,NaN
3,968451510,982860964,982454932,failed,OPHYS_7_receptive_field_mapping,NaN
4,957570596,972679204,972289789,passed,OPHYS_7_receptive_field_mapping,NaN
5,957366099,970165639,969890588,passed,OPHYS_7_receptive_field_mapping,NaN
6,957361481,969809672,969352777,passed,OPHYS_7_receptive_field_mapping,NaN
7,957024596,967873362,967401487,passed,OPHYS_7_receptive_field_mapping,NaN
8,947838486,965930948,965338397,failed,OPHYS_7_receptive_field_mapping,NaN
9,941373529,961283840,961111597,failed,OPHYS_7_receptive_field_mapping,NaN


In [165]:
for idx,row in mismatch_df.iterrows():
    if row['ophys_experiment_id'] not in my_mismatches['ophys_experiment_id'].values:
        assert False

AssertionError: 

In [166]:
row

ophys_container_id                  968451510
ophys_experiment_id                 974384470
ophys_session_id                    974019098
experiment_workflow_state              passed
stage_name_lims              OPHYS_3_images_A
stage_name_mtrain                         NaN
Name: 2, dtype: object

In [168]:
oeid = row['ophys_experiment_id']
oeid

974384470

In [169]:
stage_name_subset.query('ophys_experiment_id == @oeid')

,ophys_container_id,ophys_experiment_id,ophys_session_id,experiment_workflow_state,stage_name_lims,stage_name_mtrain
20,968451510,974384470,974019098,passed,OPHYS_3_images_A,NaN


In [144]:
oeid = 782536745
uuid = oeid_to_uuid(oeid)
pkl_path = db.get_pkl_path(uuid)
data = pd.read_pickle(pkl_path)

core_data = data_to_change_detection_core(data)
trials = create_extended_dataframe(**core_data).drop(columns=['date', 'LDT_mode'])
summary = summarize.session_level_summary(trials).iloc[0].to_dict()
summary.update({'error_on_load':0})
# vb = db.Database('visual_behavior_data')
# db.update_or_create(
#     vb['behavior_data']['summary'],
#     db.simplify_entry(summary),
#     ['behavior_session_uuid'],
#     force_write=False
# )

DatabaseError: Execution failed on sql '
    select * from well_known_files wkf
    join well_known_file_types wkft
    on wkft.id = wkf.well_known_file_type_id
    where wkf.attachable_type = 'OphysSession'
    and wkf.attachable_id in (None);
    ': column "none" does not exist
LINE 6:     and wkf.attachable_id in (None);
                                      ^


In [172]:
visb_experiments.query('ophys_experiment_id == 974384470')

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,mongo_stage_name,mongo_date,uuid,lims_stage_name,mtrain_stage_name
1291,974384470,974019098,974140453,968451510,VisualBehavior,container_qc,passed,20191030_479426_3imagesAretake,OPHYS_3_images_A,CAM2P.5,2019-10-30 15:40:18.000000,926756680,916458892,F,141.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,[Ai94(TITL-GCaMP6s)],"[Camk2a-tTA, Slc17a7-IRES2-Cre]",175,VISp,NaN,OPHYS_3_images_A,2019-10-30 16:11:37.742,5afb6da8-8432-498b-b267-06653346718a,OPHYS_3_images_A,NaN


In [125]:
data['items']['behavior']['params']

{'auto_reward_delay': 0.15,
 'free_reward_trials': 10000,
 'end_after_response': True,
 'flash_omit_probability': 0.05,
 'change_flashes_max': 12,
 'mouse_id': '479426',
 'change_time_dist': 'geometric',
 'stimulus_window': 6.0,
 'response_window': [0.15, 0.75],
 'change_flashes_min': 4,
 'catch_frequency': None,
 'min_no_lick_time': 0.0,
 'timeout_duration': 0.3,
 'change_time_scale': 0.3,
 'volume_limit': 5.0,
 'max_task_duration_min': 60.0,
 'reward_volume': 0.007,
 'end_after_response_sec': 3.5,
 'start_stop_padding': 300.0,
 'periodic_flash': [0.25, 0.5],
 'user_id': 'xana.waughman',
 'epilogue': {'params': {'frame_length': 0.03333333333333333,
   'movie_path': '//allen/programs/braintv/workgroups/neuralcoding/Saskia/Visual Stimuli 151207/Movie_TOE1.npy',
   'runs': 10,
   'size': [1920, 1080]},
  'name': 'fingerprint'},
 'stage': 'OPHYS_3_images_A',
 'auto_reward_vol': 0.005,
 'task_id': 'DoC',
 'stimulus': {'params': {'image_set': '//allen/programs/braintv/workgroups/nc-ophys/vi

In [136]:
uuids = visb_experiments[pd.isnull(visb_experiments['mongo_stage_name'])]['uuid'].values

In [137]:
from multiprocessing import Pool

In [140]:
with Pool(16) as pool:
    pool.map(rewrite_record, uuids)

Removing 41 omitted stimuli that also exist in frame log
Removing 63 omitted stimuli that also exist in frame log
Image file not found: //allen/programs/braintv/workgroups/nc-ophys/Doug/Stimulus_Code/image_dictionaries/Natural_Images_Lum_Matched_set_TRAINING_2017.07.14.pkl
Removing 50 omitted stimuli that also exist in frame log
Removing 29 omitted stimuli that also exist in frame log
/home/dougo/Code/visual_behavior_analysis/visual_behavior/utilities.py:156: RuntimeWarning:

invalid value encountered in subtract

/home/dougo/Code/visual_behavior_analysis/visual_behavior/utilities.py:156: RuntimeWarning:

invalid value encountered in subtract

/home/dougo/Code/visual_behavior_analysis/visual_behavior/utilities.py:156: RuntimeWarning:

invalid value encountered in subtract

Removing 64 omitted stimuli that also exist in frame log
Removing 43 omitted stimuli that also exist in frame log
Removing 60 omitted stimuli that also exist in frame log
Image file not found: //allen/programs/braint

failed on uuid bb1a87e7-a5b0-4094-b16c-fdfc8af88f02


got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'
got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'


failed on uuid 42dfe807-fd35-4beb-88c9-4db2e32fc858


got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'
got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'


...

failed on uuid f2923f61-df4b-4a42-a19b-bf69c5dfca06


got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'
got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'


failed on uuid 4cc81746-611b-4cd2-9d1b-d6bceb7184af


got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'
got an unexepcted shape
Traceback (most recent call last):
  File "/home/dougo/Code/visual_behavior_analysis/visual_behavior/translator/foraging2/extract.py", line 534, in behavior_items_or_top_level
    return exp_data["items"]["behavior"]
KeyError: 'behavior'


failed on uuid de904432-5231-4651-b303-1b2fdc192739


In [182]:
data['items']['behavior']['params']

{'auto_reward_delay': 0.15,
 'free_reward_trials': 10000,
 'end_after_response': True,
 'flash_omit_probability': 0.05,
 'change_flashes_max': 12,
 'mouse_id': '479426',
 'change_time_dist': 'geometric',
 'stimulus_window': 6.0,
 'response_window': [0.15, 0.75],
 'change_flashes_min': 4,
 'catch_frequency': None,
 'min_no_lick_time': 0.0,
 'timeout_duration': 0.3,
 'change_time_scale': 0.3,
 'volume_limit': 5.0,
 'max_task_duration_min': 60.0,
 'reward_volume': 0.007,
 'end_after_response_sec': 3.5,
 'start_stop_padding': 300.0,
 'periodic_flash': [0.25, 0.5],
 'user_id': 'xana.waughman',
 'epilogue': {'params': {'frame_length': 0.03333333333333333,
   'movie_path': '//allen/programs/braintv/workgroups/neuralcoding/Saskia/Visual Stimuli 151207/Movie_TOE1.npy',
   'runs': 10,
   'size': [1920, 1080]},
  'name': 'fingerprint'},
 'stage': 'OPHYS_3_images_A',
 'auto_reward_vol': 0.005,
 'task_id': 'DoC',
 'stimulus': {'params': {'image_set': '//allen/programs/braintv/workgroups/nc-ophys/vi

In [184]:
data['script']

'###############################################################################\n# Change Detection with Fingerprint\n# @author: justink\n# October 19, 2018\n###############################################################################\nimport argparse\nimport yaml\nfrom six import iteritems\nfrom camstim.change import DoCTask, DoCTrialGenerator\nfrom camstim.sweepstim import MovieStim\nfrom camstim import Window, Warp\nimport logging\n\n# Configure logging level\nlogging.basicConfig(level=logging.DEBUG,\n                    format=\'%(asctime)s - %(name)s - %(levelname)s - %(message)s\')\n\ndef load_params():\n    parser = argparse.ArgumentParser()\n    parser.add_argument("json_path", nargs="?", type=str, default="")\n\n    args, _ = parser.parse_known_args() # <- this ensures that we ignore other arguments that might be needed by camstim\n    print args\n    with open(args.json_path, \'r\') as f:\n        # we use the yaml package here because the json package loads as unicode, w